In the previous notebook (https://www.kaggle.com/code/hrishikguha/wefarm-embedding-similarity-scoring), I imported about 2000000 rows of the wefarm dataset, for initial analysis, and  computed embeddings and similarity scores with target financial phrases. In this notebook, I import the scored parquet file generated in the previous notebook, see what portion of the questions are financial, and finally, apply clustering techniques to group the financial questions thematically.

In [ ]:
import polars as pl
import os

# ---  CONFIGURATION AND LOADING ---
FINAL_SCORED_PATH = "all_10_parts_scored.parquet"
TEXT_COL = "question_content" # Column name for the question text

print(f"Attempting to load data from: {FINAL_SCORED_PATH}")

try:
    # Load the scored data
    scored_df_final = pl.read_parquet(FINAL_SCORED_PATH)
    print(f"✅ Data loaded successfully. Total rows: {scored_df_final.shape[0]}")
except Exception as e:
    print(f"❌ ERROR: Failed to load file. Ensure the file name is exactly 'all_10_parts_scored.parquet'")
    print(f"Error details: {e}")
    # Stop execution if data cannot be loaded
    raise

# Filter down to only the questions classified as financial (is_financial == True)
financial_questions_df = scored_df_final.filter(pl.col("is_financial"))

print("-" * 60)

# --- QUANTITATIVE INSIGHTS: VOLUME AND CONFIDENCE ---

total_questions = scored_df_final.shape[0]
financial_count = financial_questions_df.shape[0]
percent_financial = (financial_count / total_questions) * 100 if total_questions > 0 else 0

# Calculate summary statistics for the confidence scores (sim_fin_max)
confidence_summary = financial_questions_df.select(pl.col("sim_fin_max")).describe()


print("🎯 INITIAL INSIGHTS: VOLUME & CONFIDENCE")
print("-" * 60)
print(f"Total Questions Analyzed: {total_questions}")
print(f"Questions Flagged as Financial: {financial_count} ({percent_financial:.2f}%)")
print("\nConfidence Score (sim_fin_max) Summary for Financial Questions:")
print(confidence_summary)
print("=" * 60)


# ---  QUALITATIVE INSIGHTS: TOP 10 THEMES ---

print("\n🔍 QUALITATIVE INSIGHTS: TOP 10 HIGHEST CONFIDENCE QUESTIONS")
print("----------------------------------------------------------")

# Sort by the highest score and select the top 10
top_hits_display = financial_questions_df.sort("sim_fin_max", descending=True).head(10).select([
    pl.col(TEXT_COL),
    pl.col("sim_fin_max").round(4).alias("Sim_Score"),
])

# Use the corrected Polars configuration to ensure full text is printed
with pl.Config(tbl_cols=-1, set_tbl_width_chars=500 ):
    print(top_hits_display)

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import os
import matplotlib.pyplot as plt
# Filter down to financial questions and extract embeddings
financial_df = scored_df_final.filter(pl.col("is_financial"))
if financial_df.shape[0] == 0:
    raise ValueError("No financial questions found for clustering.")

financial_embeddings = np.stack(financial_df["embedding"].to_list())
print(f"Loaded {financial_df.shape[0]} financial questions.")

In [ ]:
# ---  OPTIMAL K DETERMINATION ---

def find_optimal_k(embeddings, k_range=(2, 10)):
    """Calculates Inertia (Elbow) and Silhouette Score for a range of K values."""
    inertias = []
    silhouette_scores = {}

    k_values = range(k_range[0], k_range[1] + 1)

    print("\nCalculating Elbow and Silhouette scores for K in range 2-10...")

    for k in k_values:
        # Run K-Means with the current K
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        clusters = kmeans.fit_predict(embeddings)

        # 1. Elbow Method Metric (Inertia)
        inertias.append(kmeans.inertia_)

        # 2. Silhouette Score Metric (Measures density/separation)
        # Skip Silhouette for K=1, as it requires at least two clusters
        if k > 1:
            score = silhouette_score(embeddings, clusters)
            silhouette_scores[k] = score
      # Plotting for visual determination
    plt.figure(figsize=(10, 4))

    plt.subplot(1, 2, 1)
    plt.plot(k_values, inertias, marker='o')
    plt.title('Elbow Method (Inertia)')
    plt.xlabel('Number of Clusters (K)')
    plt.ylabel('Inertia')

    plt.subplot(1, 2, 2)
    plt.plot(list(silhouette_scores.keys()), list(silhouette_scores.values()), marker='o')
    plt.title('Silhouette Score')
    plt.xlabel('Number of Clusters (K)')
    plt.ylabel('Silhouette Score')

    plt.tight_layout()
    plt.show()

    # Recommend the K with the highest Silhouette Score (most separated/dense clusters)
    if silhouette_scores:
        optimal_k = max(silhouette_scores, key=silhouette_scores.get)
        print(f"\nRecommended K based on highest Silhouette Score: {optimal_k}")
        return optimal_k

In [ ]:
# Execute K determination and use the result
K_CLUSTERS = find_optimal_k(financial_embeddings)

In [ ]:
# ---  FINAL CLUSTERING AND REPRODUCIBILITY FIX ---

print("-" * 60)
#print(f"Running final K-Means with K={K_CLUSTERS}...")

# Run the FINAL model with the determined K
kmeans_final = KMeans(n_clusters=8, random_state=42, n_init=10)
clusters = kmeans_final.fit_predict(financial_embeddings)

# Add the cluster label back to the Polars DataFrame
financial_df = financial_df.with_columns(
    pl.Series(name="cluster_label", values=clusters)
)

# FIX: Save the Cluster Centers for Reproducibility
CLUSTER_CENTERS_PATH = "cluster_centers_financial.npy"
np.save(CLUSTER_CENTERS_PATH, kmeans_final.cluster_centers_)
print(f"✅ Cluster Centers saved for future prediction/reproducibility: {CLUSTER_CENTERS_PATH}")
print("-" * 60)

# ---  CLUSTER ANALYSIS AND QUALITY EVALUATION ---

print("🎯 CLUSTER ANALYSIS: THEME DISCOVERY AND EVALUATION")
print("-" * 60)

# Calculate Cluster Quality Metrics
final_silhouette_score = silhouette_score(financial_embeddings, clusters)

print(f"Total Questions Clustered: {financial_df.shape[0]}")
print(f"Final Silhouette Score (Quality Metric): {final_silhouette_score:.4f} (Higher is Better)")

# Group by the new cluster label and count questions in each group
cluster_counts = financial_df.group_by("cluster_label").agg(
    pl.count().alias("Count"),
    (pl.col("sim_fin_max").mean()).round(4).alias("Avg_Sim_Score")
).sort("Count", descending=True)

print("\nCluster Sizes and Average Confidence:")
print(cluster_counts)

The translation is not always great but I decided to list the full text of 10 questions from each cluster to understand the exact themes. This gives us a basic idea. The cluster themes go as follows: "Saving", "Loans", "Cash Crops", "Government assistance, grants and subisidies", "Earnings from Farm", "Managing Farm Finance", "Farm Inputs", and generic questions about which farming and poultry uses are the best. Going through example question text(listed below) for each cluster might be useful. I have not listed cluster no.1 due to poor readability and generic nature.

In [ ]:
# --- CONFIGURATION ---
TARGET_CLUSTER = 0
N_DISPLAY = 10
TEXT_COL = "question_content"

# Filter the DataFrame for the target cluster
cluster_0_df = financial_df.filter(pl.col("cluster_label") == TARGET_CLUSTER)

# Sort by the highest similarity score (confidence) and select the top N_DISPLAY
top_10_questions = cluster_0_df.sort("sim_fin_max", descending=True).head(N_DISPLAY)

# Extract the relevant data into a list of Python dictionaries
questions_data = top_10_questions.select([TEXT_COL, "sim_fin_max", "cluster_label"]).to_dicts()

# --- MANUAL PRINTING FOR FULL TEXT ---

print(f"\n--- 🥇 Top {N_DISPLAY} Questions for Cluster {TARGET_CLUSTER} ---")
print(f"Total questions in Cluster {TARGET_CLUSTER}: {cluster_2_df.shape[0]}")
print("----------------------------------------------------------------")

# Manually print each question, ensuring full text is displayed
for i, row in enumerate(questions_data):
    # Print a separator for readability
    print("=" * 40)
    print(f"| RANK: {i+1}")
    print(f"| Score: {row['sim_fin_max']:.4f}")
    # Print the full question text
    print(f"| Question: {row[TEXT_COL]}")

print("=" * 40)

In [ ]:
# --- CONFIGURATION ---
TARGET_CLUSTER = 2
N_DISPLAY = 10
TEXT_COL = "question_content"

# Filter the DataFrame for the target cluster
cluster_2_df = financial_df.filter(pl.col("cluster_label") == TARGET_CLUSTER)

# Sort by the highest similarity score (confidence) and select the top N_DISPLAY
top_10_questions = cluster_2_df.sort("sim_fin_max", descending=True).head(N_DISPLAY)

# Extract the relevant data into a list of Python dictionaries
questions_data = top_10_questions.select([TEXT_COL, "sim_fin_max", "cluster_label"]).to_dicts()

# --- MANUAL PRINTING FOR FULL TEXT ---

print(f"\n--- 🥇 Top {N_DISPLAY} Questions for Cluster {TARGET_CLUSTER} ---")
print(f"Total questions in Cluster {TARGET_CLUSTER}: {cluster_2_df.shape[0]}")
print("----------------------------------------------------------------")

# Manually print each question, ensuring full text is displayed
for i, row in enumerate(questions_data):
    # Print a separator for readability
    print("=" * 40)
    print(f"| RANK: {i+1}")
    print(f"| Score: {row['sim_fin_max']:.4f}")
    # Print the full question text
    print(f"| Question: {row[TEXT_COL]}")

print("=" * 40)

In [ ]:
# --- CONFIGURATION ---
TARGET_CLUSTER = 3
N_DISPLAY = 10
TEXT_COL = "question_content"

# Filter the DataFrame for the target cluster
cluster_3_df = financial_df.filter(pl.col("cluster_label") == TARGET_CLUSTER)

# Sort by the highest similarity score (confidence) and select the top N_DISPLAY
top_10_questions = cluster_3_df.sort("sim_fin_max", descending=True).head(N_DISPLAY)

# Extract the relevant data into a list of Python dictionaries
questions_data = top_10_questions.select([TEXT_COL, "sim_fin_max", "cluster_label"]).to_dicts()

# --- MANUAL PRINTING FOR FULL TEXT ---

print(f"\n--- 🥇 Top {N_DISPLAY} Questions for Cluster {TARGET_CLUSTER} ---")
print(f"Total questions in Cluster {TARGET_CLUSTER}: {cluster_2_df.shape[0]}")
print("----------------------------------------------------------------")

# Manually print each question, ensuring full text is displayed
for i, row in enumerate(questions_data):
    # Print a separator for readability
    print("=" * 40)
    print(f"| RANK: {i+1}")
    print(f"| Score: {row['sim_fin_max']:.4f}")
    # Print the full question text
    print(f"| Question: {row[TEXT_COL]}")

print("=" * 40)

In [ ]:
# --- CONFIGURATION ---
TARGET_CLUSTER = 4
N_DISPLAY = 10
TEXT_COL = "question_content"

# Filter the DataFrame for the target cluster
cluster_4_df = financial_df.filter(pl.col("cluster_label") == TARGET_CLUSTER)

# Sort by the highest similarity score (confidence) and select the top N_DISPLAY
top_10_questions = cluster_4_df.sort("sim_fin_max", descending=True).head(N_DISPLAY)

# Extract the relevant data into a list of Python dictionaries
questions_data = top_10_questions.select([TEXT_COL, "sim_fin_max", "cluster_label"]).to_dicts()

# --- MANUAL PRINTING FOR FULL TEXT ---

print(f"\n--- 🥇 Top {N_DISPLAY} Questions for Cluster {TARGET_CLUSTER} ---")
print(f"Total questions in Cluster {TARGET_CLUSTER}: {cluster_2_df.shape[0]}")
print("----------------------------------------------------------------")

# Manually print each question, ensuring full text is displayed
for i, row in enumerate(questions_data):
    # Print a separator for readability
    print("=" * 40)
    print(f"| RANK: {i+1}")
    print(f"| Score: {row['sim_fin_max']:.4f}")
    # Print the full question text
    print(f"| Question: {row[TEXT_COL]}")

print("=" * 40)

In [ ]:
# --- CONFIGURATION ---
TARGET_CLUSTER = 5
N_DISPLAY = 10
TEXT_COL = "question_content"

# Filter the DataFrame for the target cluster
cluster_5_df = financial_df.filter(pl.col("cluster_label") == TARGET_CLUSTER)

# Sort by the highest similarity score (confidence) and select the top N_DISPLAY
top_10_questions = cluster_5_df.sort("sim_fin_max", descending=True).head(N_DISPLAY)

# Extract the relevant data into a list of Python dictionaries
questions_data = top_10_questions.select([TEXT_COL, "sim_fin_max", "cluster_label"]).to_dicts()

# --- MANUAL PRINTING FOR FULL TEXT ---

print(f"\n--- 🥇 Top {N_DISPLAY} Questions for Cluster {TARGET_CLUSTER} ---")
print(f"Total questions in Cluster {TARGET_CLUSTER}: {cluster_2_df.shape[0]}")
print("----------------------------------------------------------------")

# Manually print each question, ensuring full text is displayed
for i, row in enumerate(questions_data):
    # Print a separator for readability
    print("=" * 40)
    print(f"| RANK: {i+1}")
    print(f"| Score: {row['sim_fin_max']:.4f}")
    # Print the full question text
    print(f"| Question: {row[TEXT_COL]}")

print("=" * 40)

In [ ]:
# --- CONFIGURATION ---
TARGET_CLUSTER = 6
N_DISPLAY = 10
TEXT_COL = "question_content"

# Filter the DataFrame for the target cluster
cluster_6_df = financial_df.filter(pl.col("cluster_label") == TARGET_CLUSTER)

# Sort by the highest similarity score (confidence) and select the top N_DISPLAY
top_10_questions = cluster_6_df.sort("sim_fin_max", descending=True).head(N_DISPLAY)

# Extract the relevant data into a list of Python dictionaries
questions_data = top_10_questions.select([TEXT_COL, "sim_fin_max", "cluster_label"]).to_dicts()

# --- MANUAL PRINTING FOR FULL TEXT ---

print(f"\n--- 🥇 Top {N_DISPLAY} Questions for Cluster {TARGET_CLUSTER} ---")
print(f"Total questions in Cluster {TARGET_CLUSTER}: {cluster_2_df.shape[0]}")
print("----------------------------------------------------------------")

# Manually print each question, ensuring full text is displayed
for i, row in enumerate(questions_data):
    # Print a separator for readability
    print("=" * 40)
    print(f"| RANK: {i+1}")
    print(f"| Score: {row['sim_fin_max']:.4f}")
    # Print the full question text
    print(f"| Question: {row[TEXT_COL]}")

print("=" * 40)

In [ ]:
# --- CONFIGURATION ---
TARGET_CLUSTER = 7
N_DISPLAY = 10
TEXT_COL = "question_content"

# Filter the DataFrame for the target cluster
cluster_7_df = financial_df.filter(pl.col("cluster_label") == TARGET_CLUSTER)

# Sort by the highest similarity score (confidence) and select the top N_DISPLAY
top_10_questions = cluster_7_df.sort("sim_fin_max", descending=True).head(N_DISPLAY)

# Extract the relevant data into a list of Python dictionaries
questions_data = top_10_questions.select([TEXT_COL, "sim_fin_max", "cluster_label"]).to_dicts()

# --- MANUAL PRINTING FOR FULL TEXT ---

print(f"\n--- 🥇 Top {N_DISPLAY} Questions for Cluster {TARGET_CLUSTER} ---")
print(f"Total questions in Cluster {TARGET_CLUSTER}: {cluster_2_df.shape[0]}")
print("----------------------------------------------------------------")

# Manually print each question, ensuring full text is displayed
for i, row in enumerate(questions_data):
    # Print a separator for readability
    print("=" * 40)
    print(f"| RANK: {i+1}")
    print(f"| Score: {row['sim_fin_max']:.4f}")
    # Print the full question text
    print(f"| Question: {row[TEXT_COL]}")

print("=" * 40)